In [1]:
import pandas
import pathlib
import rapidfuzz
import tqdm

# load test dataset.

df = pandas.read_parquet(pathlib.Path.cwd() / 'data.parquet')
df = df[['agent_id', 'agent_label', 'work_id', 'work_label']].drop_duplicates()

# define a dataframe to store results.

results = pandas.DataFrame(columns=['agent_a_id', 'agent_b_id'])

# loop through all agents.

for subject in tqdm.tqdm([x for x in df.agent_id.unique()]):

    # reduce to the subject agent filmography, eg artistic works that they were involved with.

    subject_filmography = df.loc[df.agent_id.isin([subject])]

    # expand to include all participants who contributed to those works, so a full list of their collaborators.

    subject_community = df.loc[df.work_id.isin(subject_filmography.work_id)]
    subject_community = subject_community.loc[~subject_community.agent_id.isin([subject])]
    
    # loop through each collaborator, seeking to identify any other collaborators with a similar name.

    for f in subject_community.agent_id.unique():

        # name of participant, for matching.

        name = df.loc[df.agent_id.isin([f])]

        # all the other participants.

        collaborators = subject_community.loc[~subject_community.agent_id.isin([f])]  

        # string matching on names, using rapidfuzz library.

        string_matching = rapidfuzz.process.extract(
            name.iloc[0]['agent_label'], 
            list(collaborators.agent_label.unique()), 
            scorer=rapidfuzz.fuzz.WRatio, # different scoring models will produce different results.
            processor=rapidfuzz.utils.default_process)
        
        # reduce to name matches of equal 90 or greater levenshtein distance.

        agent_label_match = [x[0] for x in string_matching if x[1] >= 90]

        # post to results dataframe.

        collaborator = collaborators.loc[collaborators.agent_label.isin(agent_label_match)]
        for t in collaborator.agent_id.unique():
            results.loc[len(results)] = sorted([f, t])

# splice in a label for each id.

results = results.drop_duplicates()
agent_label = df[['agent_id', 'agent_label']].drop_duplicates()
for side in ['a', 'b']:
    results = pandas.merge(
        results, 
        agent_label.rename(columns={'agent_id':f'agent_{side}_id', 'agent_label': f'agent_{side}_label'}),
        on=f'agent_{side}_id',
        how='left'        
        )

# render report.

report = results[['agent_a_id', 'agent_a_label', 'agent_b_id', 'agent_b_label']]
report.to_csv(pathlib.Path.cwd() / 'report.csv', index=False)
print(len(report))
report.head(10)

100%|██████████| 32988/32988 [03:32<00:00, 155.05it/s]

1723


,agent_a_id,agent_a_label,agent_b_id,agent_b_label
0,13073,John Edwards,85782,John Edwards
1,26294,Southern Star Entertainment,85278,Southern Star Entertainment
2,13167,Vickers,14354,Vickers Films
3,35387,Showtime,69553,Showtime Australia
4,60293,SBS Independent,76657,Independent
5,11192,Carolyn Johnson,30632,Carolyn-Johnson Films
6,27489,Daniel Scharf Productions,55527,Daniel Scharf
7,12262,Greenpark Productions,15011,Green Park Productions
8,12557,Strand Films,30807,Strand
9,9021,Robert Rothols,10914,Robert Rothels
